In [1]:
import pandas as pd

In [2]:
#El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que
#acontecen en Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el
#siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud). Los
#posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin
#resolución. Las fechas se encuentran en el formato YYYY-MM-DD. Por otro lado el
#comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la
#batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un
#Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal
#tuvo una respuesta positiva (1) o negativa (0) de parte de él El sector encargado de las
#estadísticas oficiales del GCPD quiere con esta información analizar las siguientes situaciones:

In [3]:
#A. Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando
#aquellos casos en los que no participó Batman)

In [64]:
casos = pd.read_csv('casos_policiales.csv')
casos

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2019-05-25,1,nosenose,1,robo,13,43
1,2019-06-26,2,nosenose,2,robo,124,12
2,2019-05-04,3,nosenose,3,estafa,124,543
3,2019-07-23,4,nosenose,1,homicidio,123,543
4,2020-06-22,5,nosenose,2,robo,234,123
5,2019-05-15,6,nosenose,3,homicidio,123,523
6,2020-05-17,7,nosenose,1,estafa,543,23
7,2019-06-20,8,nosenose,2,robo,123,134
8,2020-07-21,9,nosenose,3,estafa,54,14
9,2020-08-22,11,nosenose,1,homicidio,132,432


In [42]:
#necesito la tasa de los casos con estado = 2

In [39]:
cantidad_casos_x_estado = casos.groupby('estado_caso').agg({'id_caso':'count'}).reset_index()
cantidad_casos_x_estado

,estado_caso,id_caso
0,1,4
1,2,3
2,3,3


In [45]:
tasa_resueltos = cantidad_casos_x_estado[cantidad_casos_x_estado['estado_caso']==2]['id_caso'] / casos['id_caso'].count()
tasa_resueltos

1    0.3
Name: id_caso, dtype: float64

In [46]:
#B. Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos
#en los que fue llamado con la batiseñal, participó en la resolución).

In [48]:
batman = pd.read_csv('batman.csv')
batman

,id_caso,respuesta
0,1,1
1,2,0
2,5,1
3,7,0
4,8,1
5,11,0


In [49]:
#primero filtro los casos en los que ayudó batman
casos_batman = batman[batman['respuesta']==1]
casos_batman

,id_caso,respuesta
0,1,1
2,5,1
4,8,1


In [50]:
#necesito los casos en los que ayudo batman y que fueron resueltos
casos_batman = casos_batman.merge(casos, how='inner', on='id_caso')
casos_batman

,id_caso,respuesta,fecha,descripcion,estado_caso,categoria,latitud,longitud
0,1,1,2018-05-25,nosenose,1,robo,13,43
1,5,1,2018-06-22,nosenose,2,robo,234,123
2,8,1,2018-06-20,nosenose,2,robo,123,134


In [53]:
cantidad_casos_batman_x_estado = casos_batman.groupby('estado_caso').agg({'id_caso':'count'}).reset_index().rename(columns={'id_caso':'cantidad'})
cantidad_casos_batman_x_estado

,estado_caso,cantidad
0,1,1
1,2,2


In [56]:
tasa_resolucion_batman = cantidad_casos_batman_x_estado[cantidad_casos_batman_x_estado['estado_caso']==2]['cantidad']/casos_batman['id_caso'].count()
tasa_resolucion_batman

1    0.666667
Name: cantidad, dtype: float64

In [57]:
#C. Indicar el mes del año pasado en el que Batman tuvo mayor participación en la
#investigación de casos.

In [65]:
#primero me quedo con los casos del año pasado
casos['fecha'] = pd.to_datetime(casos['fecha'])
casos_2019 = casos[casos['fecha'].dt.year == 2019]
casos_2019

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2019-05-25,1,nosenose,1,robo,13,43
1,2019-06-26,2,nosenose,2,robo,124,12
2,2019-05-04,3,nosenose,3,estafa,124,543
3,2019-07-23,4,nosenose,1,homicidio,123,543
5,2019-05-15,6,nosenose,3,homicidio,123,523
7,2019-06-20,8,nosenose,2,robo,123,134


In [87]:
#busco los casos en que participo batman
casos_batman_2019 = batman.merge(casos_2019, on='id_caso', how='inner')
casos_batman_2019

,id_caso,respuesta,fecha,descripcion,estado_caso,categoria,latitud,longitud
0,1,1,2019-05-25,nosenose,1,robo,13,43
1,2,0,2019-06-26,nosenose,2,robo,124,12
2,8,1,2019-06-20,nosenose,2,robo,123,134


In [88]:
#agrupo por anio
casos_batman_2019=casos_batman_2019.loc[:,['id_caso','fecha']]
casos_batman_2019['mes'] = casos_batman_2019.fecha.dt.month
casos_batman_2019

,id_caso,fecha,mes
0,1,2019-05-25,5
1,2,2019-06-26,6
2,8,2019-06-20,6


In [90]:
casos_batman_2019['cantidad_x_mes'] = 1

In [96]:
casos_batman_2019.groupby('mes').agg({'cantidad_x_mes':'sum'})['cantidad_x_mes'].idxmax()

6